# Pipelines (Parte 1)

Los pipelines son una secuencia de pasos para procesar información.

Así mismo, siguiendo este concepto, un pipeline en Scikit-Learn es una forma de aplicar secuencialmente una lista de transformaciones o predicciones a un conjunto de datos. 

En lugar de llevar a cabo la ejecución y almacenamiento de cada paso manualmente, los pipelines te permiten organizar el pre-procesamiento, extracción de características y entrenamiento en un solo lugar. Y después, puedes reutilizarlos para cuando tienes que realizar nuevas predicciones.

Esto simplifica tu código, dota de consistencia en tus proyectos y hace muy sencilla la tarea de compartir y reutilizar el código.

Los pipelines siguen exactamente la misma interfaz que ya hemos visto que comparten muchos objetos en Scikit-Learn

## La clase <code>Pipeline</code>

La clase al rededor de la que se centra todo es la clase <code>Pipeline</code>:

In [ ]:
from sklearn.pipeline import Pipeline

Esta recibe una lista de tuplas de transformadores asociados con un nombre, por ejemplo, vamos a crear un pipeline con dos pasos, uno que escale unas variables y otro que reduzca las dimensiones a un dataset – dos transformaciones que ya vimos en este curso:

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

pipeline = Pipeline([
	('scaler', StandardScaler()),
	('pca', PCA(n_components=2)),
])

Y ahora vamos a cargar unos datos para demostrar cómo es que funciona – nota que <code>X_train</code> es un una matriz de 4 columnas:

In [ ]:
from utils import load_split_iris

X_train, X_test, y_train, y_test = load_split_iris()

Con esto, ya podemos entrenar a nuestro pipeline:

In [ ]:
pipeline.fit(X_train)

Después podemos transformar nuestros dos conjuntos de datos – si ves los valores resultantes, verás que ahora solo son dos dimensiones gracias a la reducción de dimensiones que agregamos:

In [ ]:
X_train_transformed = pipeline.transform(X_train)
X_test_transformed = pipeline.transform(X_test)

Y ahora sí, estos datos podemos usarlos en un clasificador, por ejemplo:

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train_transformed, y_train)
y_pred = lr.predict(X_test_transformed)
score = lr.score(X_test_transformed, y_test)
print(f'Test accuracy: {score:.2f}')

¿Excelente no? ahora ya no tenemos que preocuparnos por tener que guardar el scaler y pca por separado. Y ahora podemos utilizar el mismo pipeline cuando pongamos nuestros datos en producción…

## Los pipelines como modelos de machine learning

Pero, ¿qué me dirías si te dijera que podemos incluir nuestro modelo como parte del pipeline en lugar de tenerlo por separado?

 Vamos a definir exactamente eso:

In [ ]:
pipeline = Pipeline([
	('scaler', StandardScaler()),
	('pca', PCA(n_components=2)),
	('lr', LogisticRegression()),
])

pipeline.fit(X_train, y_train)

Así como lo vez, el último paso de un <code>Pipeline</code> puede ser un modelo de machine learning. Y luego lo podemos utilizar para predecir nuevos valores:

In [ ]:
y_pred = pipeline.predict(X_test)
score = pipeline.score(X_test, y_test)
print(f'Test accuracy: {score:.2f}')

## Son compatibles con otras herramientas de Scikit-Learn

Los <code>Pipelines</code> también son compatibles con otras herramientas disponibles en Scikit-Learn, por ejemplo las herramientas de validación cruzada que ya vimos previamente:

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score

pipeline = Pipeline([
	('scaler', StandardScaler()),
	('pca', PCA(n_components=2)),
	('lr', LogisticRegression()),
])

cv = 5
cv_scores = cross_val_score(pipeline, X_train, y_train, cv=cv)

# Mostrar los resultados
print(f'Scores de validación cruzada ({cv} folds): {cv_scores}')
print(f'Score promedio: {np.mean(cv_scores):0.2f}')

Y también con la búsqueda de hiperparámetros:

In [ ]:
pipeline = Pipeline([
    ('scaler', StandardScaler()), # Paso 1: Escalar los datos
    ('pca', PCA()),               # Paso 2: Reducción de dimensionalidad
    ('lr', LogisticRegression()), # Paso 3: Modelo de regresión logística
])

param_grid = {
    'pca__n_components': [1, 2, 3],
    'lr__penalty': ['l1', 'l2', 'elasticnet', None],
    'lr__C': np.logspace(-3, 3, 7),
}

La peculiaridad está en cómo definimos la cuadrícula de parámetros, tienes que ponerle el nombre con el que asociaste el transformador seguido de dos guiones bajos, seguido por el nombre del argumento.

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)

In [ ]:
# Mostrar los resultados
print(f'Mejores parámetros: {grid_search.best_params_}')
print(f'Mejor puntaje: {grid_search.best_score_:.2f}')